In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xlrd
list_of_period = [5,10,15,20,25,30]
list_of_k = [1,1.5,2]

In [2]:
# Read the data from the Excel file
xls_file = '/home/vishi/bolinger/cumul_ohlc.xls'
xls = xlrd.open_workbook(xls_file)
sheet = xls.sheet_by_index(0)
# Extract the data
closing_prices = []
for row in range(1, sheet.nrows):
    closing_prices.append(sheet.cell_value(row, 4))
    


In [3]:
def get_avg_std(prices, period):
    avg = np.zeros(len(prices))
    std = np.zeros(len(prices))
    for i in range(period, len(prices)):
        avg[i] = np.mean(prices[i-period:i])
        # sample std
        std[i] = np.std(prices[i-period:i], ddof=1)
    return avg, std



In [4]:
def get_bollinger_bands(prices, period, k):
    avg, std = get_avg_std(prices, period)
    upper_band = avg + k * std
    lower_band = avg - k * std
    return avg, upper_band, lower_band
def profit_basic(prices,avg, upper_band, lower_band):
    flag = 0
    profit = 0
    current_price = 0
    list_of_profit = []
    for i in range(len(prices)):
        if flag == 0:
            if prices[i] < lower_band[i]:
                flag = 1
                current_price = prices[i]
            elif prices[i] > upper_band[i]:
                flag = -1
                current_price = prices[i]
        elif flag == 1:
            if prices[i] > avg[i]:
                flag = 0
                profit += prices[i] - current_price
        elif flag == -1:
            if prices[i] < avg[i]:
                flag = 0
                profit += current_price - prices[i]
        list_of_profit.append(profit)
    if flag == 1:
        profit += prices[-1] - current_price
    elif flag == -1:
        profit += current_price - prices[-1]
    list_of_profit.append(profit)
    return list_of_profit

def plot_profit(list_of_profit, period, k):
    plt.plot(list_of_profit)
    plt.title(f'Profit with period {period} and k {k}')
    plt.xlabel('Time')
    plt.ylabel('Profit')
    plt.show()

list_of_profits_of_diff_strategies = []
for period in list_of_period:
    list_to_append = []
    for k in list_of_k:
        avg, upper_band, lower_band = get_bollinger_bands(closing_prices, period, k)
        list_of_profit = profit_basic(closing_prices, avg, upper_band, lower_band)
        # plot_profit(list_of_profit, period, k)
        list_to_append.append(list_of_profit)
    list_of_profits_of_diff_strategies.append(list_to_append)


In [5]:
reshaped_list_of_profits = []
for i in range(3):
    for j in range(6):
        reshaped_list_of_profits.append(list_of_profits_of_diff_strategies[j][i])

        

In [6]:
# find the returns of every minute of every strategy
list_of_returns = []
for i in range(len(reshaped_list_of_profits)):
    list_of_returns.append(np.diff(reshaped_list_of_profits[i]))



In [7]:
# now i want to use test for superior predictability ability on all stratergies 

from arch.bootstrap import SPA

# create a list of all the returns
list_of_all_returns_np = np.array(list_of_returns).T

benchmark_returns = list_of_all_returns_np[:, 0]
stratergy_returns = list_of_all_returns_np[:, 1:]

diffs = stratergy_returns - benchmark_returns[:, None]

benchmark_losses = -benchmark_returns
stratergy_losses = -stratergy_returns

spa = SPA(benchmark=benchmark_losses,
    models=stratergy_losses,
    reps=1000,
    block_size=100,  
    bootstrap="stationary",
    studentize=True,
    nested=False,
    seed=42
)
spa.compute()




In [8]:
print("\nBetter models than benchmark (strategy 0):")
print(spa.better_models(pvalue=0.01, pvalue_type="consistent"))



Better models than benchmark (strategy 0):
[15]


In [9]:
def better_model_than_benchmark(i,list_of_all_returns_np):
    benchmark_returns = list_of_all_returns_np[:, i]
    # everything else
    stratergy_returns = list_of_all_returns_np[:, i+1:]
    benchmark_losses = -benchmark_returns
    stratergy_losses = -stratergy_returns
    spa = SPA(
        benchmark=benchmark_losses,
        models=stratergy_losses,
        reps=1000,
        block_size=100,  
        bootstrap="stationary",
        studentize=True,
        nested=False,
        seed=42
    )
    spa.compute()
    return spa

for i in range(len(reshaped_list_of_profits)-1):
    spa = better_model_than_benchmark(i, list_of_all_returns_np)
    print(f"\nBetter models than benchmark (strategy {i}):")
    print(spa.better_models(pvalue=0.05, pvalue_type="consistent")+i)

    


Better models than benchmark (strategy 0):
[ 4  8  9 10 14 15 16]

Better models than benchmark (strategy 1):
[]

Better models than benchmark (strategy 2):
[]

Better models than benchmark (strategy 3):
[]

Better models than benchmark (strategy 4):
[]

Better models than benchmark (strategy 5):
[]

Better models than benchmark (strategy 6):
[ 9 10 14 15 16]

Better models than benchmark (strategy 7):
[]

Better models than benchmark (strategy 8):
[]

Better models than benchmark (strategy 9):
[]

Better models than benchmark (strategy 10):
[]

Better models than benchmark (strategy 11):
[]

Better models than benchmark (strategy 12):
[14 15 16]

Better models than benchmark (strategy 13):
[]

Better models than benchmark (strategy 14):
[]

Better models than benchmark (strategy 15):
[]

Better models than benchmark (strategy 16):
[]


In [10]:
print(type(spa.better_models(pvalue=0.05, pvalue_type="consistent")))

<class 'numpy.ndarray'>


In [11]:
def better_model_than_benchmark_list(i, list_of_returns):
    benchmark_returns = np.array(list_of_returns[i]).T
    # everything else
    stratergy_returns =np.array(list_of_returns[i+1:]).T
    
    benchmark_losses = -benchmark_returns
    stratergy_losses = -stratergy_returns
    spa = SPA(benchmark=benchmark_losses,
        models=stratergy_losses,
        reps=1000,
        block_size=100,  
        bootstrap="stationary",
        studentize=True,
        nested=False,
        seed=42
    )
    spa.compute()
    return spa

for i in range(len(reshaped_list_of_profits)):
    better_model_than_benchmark_list(i, list_of_returns)
    print(f"\nBetter models than benchmark (strategy {i}):")
    print(spa.better_models(pvalue=0.05, pvalue_type="consistent"))




Better models than benchmark (strategy 0):
[]

Better models than benchmark (strategy 1):
[]

Better models than benchmark (strategy 2):
[]

Better models than benchmark (strategy 3):
[]

Better models than benchmark (strategy 4):
[]

Better models than benchmark (strategy 5):
[]

Better models than benchmark (strategy 6):
[]

Better models than benchmark (strategy 7):
[]

Better models than benchmark (strategy 8):
[]

Better models than benchmark (strategy 9):
[]

Better models than benchmark (strategy 10):
[]

Better models than benchmark (strategy 11):
[]

Better models than benchmark (strategy 12):
[]

Better models than benchmark (strategy 13):
[]

Better models than benchmark (strategy 14):
[]

Better models than benchmark (strategy 15):
[]

Better models than benchmark (strategy 16):
[]


ValueError: operands could not be broadcast together with shapes (23625,1) (0,1) 